<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# The IBM Q Provider 

In Qiskit we have an interface for backends and jobs that is useful for running circuits and extending to third-party backends. In this tutorial, we will review the core components of Qiskit’s base backend framework, using the IBM Q Provider as an example.

The interface has three main component: providers, backends, and jobs:

- providers: access backends and provides backend objects
- backends: run the quantum circuit
- jobs: keep track of the submitted job

## The Provider

The IBM Q Provider is an entity that provides access to groups of different backends (for example, backends available through the open IBM Q Experience, or specific groups in the IBM Q Network).

The IBM Q Provider main point of entry is as `qiskit.IBMQ`. This is an object that allows obtaining `provider` instances, corresponding to your IBM Q Experience capabilities:

- `providers()`: returns all the provider objects in your account.
- `get_provider(hub, group, project)`: returns the specified provider.

Additionally, it has some extra functions for handling administrative tasks. The credentials can be saved to disk or used in a session and never saved. 

- `enable_account(token, url)`: enable the account in the current session
- `disable_account(**kwargs)`: disable the accounts from current session
- `save_account(token)`: save the account to disk
- `delete_account(**kwargs)`: delete the account from disk
- `load_account(**kwargs)`: load previously-saved account into session
- `active_account()`: list the account active in this session
- `stored_account()`: list the account saved to disk

The `provider` objects obtained through `get_provider()` inherit from BaseProvider and implement the methods:

- `backends()`: returns all backend objects known to the provider.
- `get_backend(name)`: returns the named backend.


<div class="alert alert-block alert-info">
<b>Note:</b> The use of `provider` instances is the default way of retrieving backends from Qiskit 0.11 onwards - if you have been using earlier versions of Qiskit, check the "Updating from previous versions" section for more detailed instructions on updating and using the different options.</div>
</div>

In [1]:
from qiskit import IBMQ 

IBMQ.providers()

[]

Here we see that there are no providers. This is because no accounts have been loaded.

Let's start fresh and delete any accounts on disk. If no accounts are on disk, this will error

In [2]:
IBMQ.delete_account()

verify that there are no accounts stored now

In [3]:
IBMQ.stored_account()

[]

To enable an account (useful for one-off use, or if you don't want to save to disk)

In [ ]:
# Change this to be your API Token
my_api_token = 'CHANGE_THIS_TO_YOUR_API_TOKEN'
provider = IBMQ.enable_account(my_api_token)

The `IBMQ.enable_account()` and `IBMQ.load_account()` methods will automatically return the provider for the open IBM Q Experience, for convenience. To see what providers you have available

In [2]:
IBMQ.providers()

[<AccountProvider for IBMQ(ibm-q, open, main)>]

and backends which are available in the default provider

In [7]:
provider.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

Disable that account (so we go back to no accounts active)

In [8]:
IBMQ.disable_account()

Now no providers are available

In [9]:
IBMQ.providers()

[]

For convenience, you can save your account to disk:

In [10]:
IBMQ.save_account(my_api_token, overwrite=True)

Now it should show up as present on disk

In [11]:
# uncomment to print to screen (it will show your token and url)
# IBMQ.stored_account()

but no account active in current session yet

In [12]:
IBMQ.active_account()

[]

now load up the account stored to disk

In [14]:
provider = IBMQ.load_account()

If you have access to multiple hubs, you can obtain a provider for each of them using `.get_provider()`:

In [15]:
provider_1 = IBMQ.get_provider(hub='open')
provider_1.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

In [16]:
provider_2 = IBMQ.get_provider(hub='ibm-q-internal')
provider_2.backends()

[<IBMQBackend('ibmq_20_tokyo') from IBMQ(ibm-q-internal, yrk, main)>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(ibm-q-internal, yrk, main)>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ(ibm-q-internal, yrk, main)>]

## Filtering the backends

You may also optionally filter the set of returned backends, by passing arguments that query the backend's `configuration`, `status`, or `properties`. The filters are passed by conditions, and for more general filters you can make advanced functions using the lambda function.

As a first example: only return currently operational devices

In [21]:
provider.backends(operational=True, simulator=False)

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>]

only return backends that are real devices, have at most 5 qubits, and are operational

In [22]:
provider.backends(filters=lambda x: x.configuration().n_qubits <= 5 and 
              not x.configuration().simulator and x.status().operational==True)

[<IBMQBackend('ibmqx4') from IBMQ()>, <IBMQBackend('ibmqx2') from IBMQ()>]

Filter: show the least busy device (in terms of pending jobs in the queue)

In [23]:
from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits == 5 and
                                                       not x.configuration().simulator)
least_busy(small_devices)

<IBMQBackend('ibmqx4') from IBMQ()>

The above filters can be combined as desired.

If you just want to get an instance of a particular backend, you can use the `get_backend()` method.

In [24]:
provider.get_backend('ibmq_16_melbourne')

<IBMQBackend('ibmq_16_melbourne') from IBMQ()>

## The backend

Backends represent either a simulator or a real quantum computer, and are responsible for running quantum circuits and returning results. Their `run` method takes in a `qobj` as input, which is a quantum object and the result of the compilation process; it returns a BaseJob object. This object allows asynchronous running of jobs for retrieving results from a backend when the job is completed.

At a minimum, backends use the following methods, inherited from BaseBackend:


- `provider` - returns the provider of the backend.
- `name()` - gets the name of the backend.
- `status()` - gets the status of the backend.
- `configuration()` - gets the configuration of the backend.
- `properties()` - gets the properties of the backend.
- `run()` - runs a qobj on the backend.

For remote backends, they must also support

- `jobs()` - returns a list of previous jobs executed by this user on this backend.
- `retrieve_job()` - returns a job by a job_id.

In future updates they will introduce the following commands

- `defaults()` - gives a data structure of typical default parameters.
- `schema()` - gets a schema for the backend

There are some IBM Q-only functions 

- `hub`  - returns the IBM Q hub for this backend.
- `group` - returns the IBM Q group for this backend.
- `project` - returns the IBM Q project for this backend.

In [25]:
backend = least_busy(small_devices)

Let's start with the `backend.provider`, which returns a provider object

In [26]:
backend.provider

<bound method BaseBackend.provider of <IBMQBackend('ibmqx4') from IBMQ()>>

Next is the `name()`, which returns the name of the backend

In [27]:
backend.name()

'ibmqx4'

Next let's look at the `status()`:

    operational lets you know that the backend is taking jobs
    pending_jobs lets you know how many jobs are in the queue

In [28]:
backend.status()

BackendStatus(backend_name='ibmqx4', backend_version='1.0.0', operational=True, pending_jobs=7, status_msg='active')

The next is `configuration()`

In [29]:
backend.configuration()

QasmBackendConfiguration(allow_q_object=True, backend_name='ibmqx4', backend_version='1.0.0', basis_gates=['u1', 'u2', 'u3', 'cx', 'id'], conditional=False, coupling_map=[[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]], credits_required=True, description='5 qubit device', gates=[GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='id', parameters=[], qasm_def='gate id q { U(0,0,0) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u1', parameters=['lambda'], qasm_def='gate u1(lambda) q { U(0,0,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u2', parameters=['phi', 'lambda'], qasm_def='gate u2(phi,lambda) q { U(pi/2,phi,lambda) q; }'), GateConfig(coupling_map=[[0], [1], [2], [3], [4]], name='u3', parameters=['theta', 'phi', 'lambda'], qasm_def='u3(theta,phi,lambda) q { U(theta,phi,lambda) q; }'), GateConfig(coupling_map=[[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]], name='cx', parameters=[], qasm_def='gate cx q1,q2 { CX q1,q2; }')], local=Fa

The next is `properties()` method

In [30]:
backend.properties()

BackendProperties(backend_name='ibmqx4', backend_version='1.0.0', gates=[Gate(gate='u1', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='', value=0.0)], qubits=[0]), Gate(gate='u2', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='', value=0.0007726307293453583)], qubits=[0]), Gate(gate='u3', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='', value=0.0015452614586907165)], qubits=[0]), Gate(gate='u1', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='', value=0.0)], qubits=[1]), Gate(gate='u2', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='', value=0.0017172252255832077)], qubits=[1]), Gate(gate='u3', parameters=[Nduv(date=datetime.datetime(2019, 5, 5, 9, 55, 18, tzinfo=tzutc()), name='gate_error', unit='',

The next is `hub`, `group`, and `project`. For the IBM Q experience these will return `None`

In [31]:
backend.hub

In [32]:
backend.group

In [33]:
backend.project

To see your last 5 jobs run on the backend, use the `jobs()` method of that backend

In [34]:
for ran_job in backend.jobs(limit=5):
    print(str(ran_job.job_id()) + " " + str(ran_job.status()))

5ccf80446443ec007394c5c8 JobStatus.DONE
5ccf80389bfaab0074c5cf78 JobStatus.CANCELLED
5ccf7ff47b9d00006d60b9cd JobStatus.DONE
5ccf7d36557a5600718c5793 JobStatus.DONE
5ccf450535c8b100714dcc53 JobStatus.DONE


Then the job can be retreived using the `retrieve_job(job_id())` method

In [35]:
job = backend.retrieve_job(ran_job.job_id())

## The Job object

Job instances can be thought of as the “ticket” for a submitted job. They find out the execution’s state at a given point in time (for example, if the job is queued, running, or has failed), and allow control over the job. They have the following methods:

- `status()` - returns the status of the job.
- `backend()` - returns the backend the job was run on.
- `job_id()` - gets the job_id.
- `cancel()` - cancels the job.
- `result()` - gets the results from the circuit run.

IBM Q-only functions 

- `creation_date()` - gives the date at which the job was created.
- `queue_position()` - gives the position of the job in the queue.
- `error_message()` - gives the error message of failed jobs.

Let's start with the `status()`. This returns the job status and a message

In [36]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

To get a backend object from the job, use the `backend()` method

In [37]:
backend_temp = job.backend()
backend_temp

<IBMQBackend('ibmqx4') from IBMQ()>

To get the job_id use the `job_id()` method

In [38]:
job.job_id()

'5ccf450535c8b100714dcc53'

To get the result from the job, use the `result()` method

In [39]:
result = job.result()
counts = result.get_counts()
print(counts)

{'011': 25, '000': 452, '100': 22, '001': 19, '110': 77, '010': 21, '101': 67, '111': 341}


If you want to check the creation date, use `creation_date()`

In [40]:
job.creation_date()

'2019-05-05T20:18:13.060Z'

Let's make an active example

In [41]:
from qiskit import *
from qiskit.compiler import transpile, assemble

In [42]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

To compile this circuit for the backend, use the compile function. It will make a qobj (quantum object) that can be run on the backend using the `run(qobj)` method. 

In [43]:
qobj = assemble(transpile(circuit, backend=backend), shots=1024)
job = backend.run(qobj)

The status of this job can be checked with the `status()` method

In [44]:
job.status()

<JobStatus.INITIALIZING: 'job is being initialized'>

If you made a mistake and need to cancel the job, use the `cancel()` method.

In [45]:
import time
#time.sleep(10)

job.cancel()

False

The `status()` will show that the job cancelled. 

In [46]:
job.status()

<JobStatus.INITIALIZING: 'job is being initialized'>

To rerun the job and set up a loop to check the status and queue position, use the `queue_position()` method. 

In [47]:
job = backend.run(qobj)

In [48]:
from qiskit.tools.monitor import job_monitor
job_monitor(job)
result = job.result()

Job Status: job has successfully run


In [49]:
counts = result.get_counts()
print(counts)

{'011': 45, '000': 45, '100': 147, '001': 100, '110': 24, '010': 18, '101': 599, '111': 46}


## Updating from previous versions

Since July 2019 (and with `Qiskit` version `0.11`), the IBM Q Provider defaults to using the new [IBM Q Experience](https://quantum-computing.ibm.com), which supersedes the legacy Quantum Experience and Qconsole.

If you have credentials for the legacy Quantum Experience or Qconsole stored in disk, you can make use of `IBMQ.update_account()` helper. This helper will read your current credentials stored in disk and attempt to convert them:


In [1]:
# IBMQ.update_account()

You can find more information on how to update your programs in the [README.md](https://github.com/Qiskit/qiskit-ibmq-provider/blob/master/README.md#updating-to-the-new-ibm-q-experience) file in the provider repository.